In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.to_datetime(pd.Series(['Jan 1 2001','01-04-2001', '2, 4, 2000','2001 2 5', 'May 2003', '2003/1/30','01.03.2005']))
#European sysle day first
pd.to_datetime(['14-01-2012', '01-14-2012'], dayfirst=True)
##mix
"""pd.to_datetime(pd.Series(
    ['Jan 1 2001',
     '01-04-2001', 
     '2, 4, 2000',
     '2001 2 5', 
     'May 2003', 
     '2003/1/30',
     '01.03.2005 10:00']),
               dayfirst=True)"""


"pd.to_datetime(pd.Series(\n    ['Jan 1 2001',\n     '01-04-2001', \n     '2, 4, 2000',\n     '2001 2 5', \n     'May 2003', \n     '2003/1/30',\n     '01.03.2005 10:00']),\n               dayfirst=True)"

In [23]:
df = pd.DataFrame({'year': [2015, 2016],'month': [2, 3],'day': [4, 5],'hour': [2, 3]})
pd.to_datetime(df)
pd.to_datetime(df[['year','month','day']])
##epock timestamp
pd.to_datetime([1349720105, 1349806505, 1349892905,1349979305, 1350065705], unit='s')

DatetimeIndex(['2012-10-08 18:15:05', '2012-10-09 18:15:05',
               '2012-10-10 18:15:05', '2012-10-11 18:15:05',
               '2012-10-12 18:15:05'],
              dtype='datetime64[ns]', freq=None)

In [24]:
##timestaps to epock
stamps = pd.date_range('2012-10-08 18:15:05', periods=4, freq='D')
(stamps - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

Int64Index([1349720105, 1349806505, 1349892905, 1349979305], dtype='int64')

In [25]:
##multi index datetime slicing
dft2 = pd.DataFrame(
    np.random.randn(20, 1), columns=['A'], 
    index=pd.MultiIndex.from_product(
        [pd.date_range(
            '20130101', 
            periods=10,
            freq='12H'
        ),
         ['a', 'b']]
    )
) ;dft2
idx = pd.IndexSlice
dft2 = dft2.swaplevel(0, 1).sort_index()
dft2.loc[idx[:, '2013-01-05'], :]

A
a 2013-01-05 00:00:00  0.767856
  2013-01-05 12:00:00  0.067074
b 2013-01-05 00:00:00 -0.295545
  2013-01-05 12:00:00 -1.001557

In [26]:
series_minute = pd.Series([1,2,3],pd.DatetimeIndex(['2001-12-12 23:57:00','2001-12-12 23:58:00','2001-12-12 23:59:00']))
dft_minute = pd.DataFrame({'a':np.arange(0,3), 'b':np.arange(3,6)}, index= series_minute.index)
## exact matching is error -> dft_minute['2001-12-12 23:57:00']
dft_minute.loc['2001-12-12 23:57:00']

a    0
b    3
Name: 2001-12-12 23:57:00, dtype: int64

In [27]:
from pandas.tseries.offsets import CustomBusinessDay
from datetime import datetime
weekmask_egypt = 'Sun Mon Tue Wed Thu'
holidays = ['2012-05-01', datetime(2013, 5, 1), np.datetime64('2014-05-01')]
bday_egypt = CustomBusinessDay(holidays=holidays, weekmask=weekmask_egypt)
dt = datetime(2013, 4, 30)
dt + 2 * bday_egypt ## two time a business day ahead
##4/30 ->Tues 5/1 ->holiday 5/2-> Thursday(First bday) 5/3-> Egypt weekend 5/5-> Sunday(Egypt bday)

Timestamp('2013-05-05 00:00:00')

In [28]:
dtt = pd.Series(datetime(2013,5,2))
dtt.dt.dayofweek

0    3
dtype: int64

In [29]:
dts = pd.date_range(dt, periods=5, freq=bday_egypt)
pd.Series(dts.weekday, dts).map(pd.Series('Mon Tue Wed Thu Fri Sat Sun'.split()))

2013-04-30    Tue
2013-05-02    Thu
2013-05-05    Sun
2013-05-06    Mon
2013-05-07    Tue
Freq: C, dtype: object

In [30]:
from pandas.tseries.holiday import Holiday, USMemorialDay,AbstractHolidayCalendar, nearest_workday, MO,DateOffset
from pandas.tseries.offsets import CDay
class ExampleCalendar(AbstractHolidayCalendar):
     rules = [
         USMemorialDay,
         Holiday('July 4th', month=7, day=4, observance=nearest_workday),
         Holiday('Columbus Day', month=10, day=1,
                 offset=DateOffset(weekday=MO(2))), #same as 2*Week(weekday=2)
     ]
cal = ExampleCalendar()
cal.holidays(datetime(2012, 1, 1), datetime(2012, 12, 31))
pd.DatetimeIndex(start='7/1/2012', end='7/10/2012', freq=CDay(calendar=cal)).to_pydatetime()
offset = CustomBusinessDay(calendar=cal)
datetime(2012, 7, 3) + offset

Timestamp('2012-07-05 00:00:00')

In [31]:
from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory,USLaborDay
cal = get_calendar('ExampleCalendar')
new_cal = HolidayCalendarFactory('NewExampleCalendar', cal, USLaborDay)
new_cal.rules

[Holiday: Columbus Day (month=10, day=1, offset=<DateOffset: weekday=MO(+2)>),
 Holiday: July 4th (month=7, day=4, observance=<function nearest_workday at 0x11403c950>),
 Holiday: MemorialDay (month=5, day=31, offset=<DateOffset: weekday=MO(-1)>),
 Holiday: Labor Day (month=9, day=1, offset=<DateOffset: weekday=MO(+1)>)]

In [32]:
index = pd.DatetimeIndex(pd.date_range('2018/01/01',periods = 10))
ts = pd.Series(np.random.randn(len(index)),index=index)
ts = pd.date_range('2011-01-31', '2011-12-30', freq='BM')
ts.shift(5, freq='BM')#shifing data

DatetimeIndex(['2011-06-30', '2011-07-29', '2011-08-31', '2011-09-30',
               '2011-10-31', '2011-11-30', '2011-12-30', '2012-01-31',
               '2012-02-29', '2012-03-30', '2012-04-30', '2012-05-31'],
              dtype='datetime64[ns]', freq='BM')

In [33]:
rng2 = pd.date_range('1/1/2012', end='3/31/2012', freq='D')
ts2 =pd.Series(range(len(rng2)),index=rng2)
ts2[:2].resample('6H').asfreq().ffill(limit=2)

2012-01-01 00:00:00    0.0
2012-01-01 06:00:00    0.0
2012-01-01 12:00:00    0.0
2012-01-01 18:00:00    NaN
2012-01-02 00:00:00    1.0
Freq: 6H, dtype: float64

In [34]:
df = pd.DataFrame({'date': pd.date_range('2015-01-01', freq='W', periods=5),
                   'a': np.arange(5)},
                  index=pd.MultiIndex.from_arrays([
                      [1,2,3,4,5],
                      pd.date_range('2015-01-01', freq='W', periods=5)],
                      names=['v','d']));
df.resample('M', on='date').sum() #df.resample('M', level='d').sum()

,a
date,
2015-01-31,6
2015-02-28,4


In [35]:
a = pd.Period(freq='D', year=2001, month=1, day=1)
a.strftime('%d-%b-%Y')
a.strftime('%b. %d, %Y was a %A')

'Jan. 01, 2001 was a Monday'

In [36]:
pd.PeriodIndex(start='2015/1/1', periods=10, freq='D')
pd.PeriodIndex(start=pd.Period('2017Q1', freq='Q'), end=pd.Period('2017Q2', freq='Q'), freq='M')

PeriodIndex(['2017-03', '2017-04', '2017-05', '2017-06'], dtype='period[M]', freq='M')

In [37]:
pi = pd.period_range('2016-01-01', periods=3, freq='M')
pi.astype('datetime64[ns]') # convert to DatetimeIndex

DatetimeIndex(['2016-01-01', '2016-02-01', '2016-03-01'], dtype='datetime64[ns]', freq='MS')

In [38]:
dti = pd.date_range('2011-01-01', freq='M', periods=3)
dti.astype('period[M]')# convert to PeriodIndex

PeriodIndex(['2011-01', '2011-02', '2011-03'], dtype='period[M]', freq='M')

In [40]:
df = pd.DataFrame(pd.date_range('2014-01-01', freq='2w', periods=12))
df[1] = pd.DatetimeIndex(df[0]).to_period('M')
df[2] = 2; df

,0,1,2
0,2014-01-05,2014-01,2
1,2014-01-19,2014-01,2
2,2014-02-02,2014-02,2
3,2014-02-16,2014-02,2
4,2014-03-02,2014-03,2
5,2014-03-16,2014-03,2
6,2014-03-30,2014-03,2
7,2014-04-13,2014-04,2
8,2014-04-27,2014-04,2
9,2014-05-11,2014-05,2


In [41]:
df.set_index([0, 1], inplace=True)
df.index.get_level_values(0) 

DatetimeIndex(['2014-01-05', '2014-01-19', '2014-02-02', '2014-02-16',
               '2014-03-02', '2014-03-16', '2014-03-30', '2014-04-13',
               '2014-04-27', '2014-05-11', '2014-05-25', '2014-06-08'],
              dtype='datetime64[ns]', name=0, freq=None)

In [42]:
p = pd.Period('2011')
p.asfreq('M','s')
p.asfreq('M','e')

Period('2011-12', 'M')

In [43]:
s = pd.Series([20121231, 20141130, 99991231])
def conv(x):
    return pd.Period(year = x // 10000, month = x//100 % 100, day = x%100, freq='D')
s.apply(conv)

0   2012-12-31
1   2014-11-30
2   9999-12-31
dtype: object

In [48]:
span = pd.PeriodIndex(s.apply(conv)) ;span

PeriodIndex(['2012-12-31', '2014-11-30', '9999-12-31'], dtype='period[D]', freq='D')